# VacationPy

## Imports

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

## Storing results as DataFrame

In [10]:
cities_df = pd.read_csv("../WeatherPy/city_weather_data.csv")
cities_df.head()

,Latitude,Longitude,City,Country,Max Temperature (F),Humidity,Cloudiness,Wind Speed (mph),Date
0,-14.936039,13.322562,lubango,ao,69.80,56.0,0.0,3.44,1610528669
1,39.658409,-48.245995,torbay,ca,28.99,93.0,75.0,8.05,1610528850
2,-89.979413,37.550464,port elizabeth,za,71.60,83.0,90.0,13.80,1610528788
3,-35.580137,165.168906,ahipara,nz,65.88,84.0,10.0,6.15,1610528850
4,-63.583940,-38.583740,ushuaia,ar,41.00,75.0,0.0,8.05,1610528649
...,...,...,...,...,...,...,...,...,...
500,50.440424,-110.868744,medicine hat,ca,45.00,65.0,1.0,14.97,1610528977
501,8.361826,124.960650,maluko,ph,80.01,85.0,100.0,5.01,1610528978
502,56.959632,1.215939,boddam,gb,35.60,80.0,96.0,10.36,1610528978
503,16.191234,-76.291348,morant bay,jm,75.20,78.0,20.0,8.05,1610528978


## Humidity Heatmap

In [11]:
# Create a heat map that displays the humidity for every city from the part I of the homework.

# Configuring gmaps with API key
gmaps.configure(api_key=g_key)

# Storing Latitude and Longitude values into  locations 
locations = cities_df[["Latitude", "Longitude"]]

# Storing Humidity as the weight
humidity = cities_df["Humidity"]

In [36]:
# Create the map and a humidity Heatmap layer
fig = gmaps.figure(center=(20, 0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

## Specific Weather Criteria

In [39]:
cities_df.dtypes


Latitude               float64
Longitude              float64
City                    object
Country                 object
Max Temperature (F)    float64
Humidity               float64
Cloudiness             float64
Wind Speed (mph)       float64
Date                     int64
dtype: object

In [149]:
# Narrowing down the DataFrame to find ideal weather conditions: 

ideal_weather = cities_df.loc[((cities_df["Max Temperature (F)"] >= 65) & 
                               (cities_df["Max Temperature (F)"] <= 95) & 
                               (cities_df["Wind Speed (mph)"] <= 18) & 
                               (cities_df["Cloudiness"] <= 15)), :] 

ideal_weather
                               
                            

,Latitude,Longitude,City,Country,Max Temperature (F),Humidity,Cloudiness,Wind Speed (mph),Date
0,-14.936039,13.322562,lubango,ao,69.80,56.0,0.0,3.44,1610528669
3,-35.580137,165.168906,ahipara,nz,65.88,84.0,10.0,6.15,1610528850
5,-56.473162,104.629318,busselton,au,81.00,36.0,0.0,1.01,1610528771
10,-53.198595,-17.768423,arraial do cabo,br,75.20,88.0,0.0,12.66,1610528852
19,-33.583648,130.542010,flinders,au,73.99,79.0,10.0,8.99,1610528854
45,34.992028,-153.096680,kapaa,us,73.40,83.0,1.0,6.91,1610528753
53,-65.254553,-129.726240,rikitea,pf,77.76,79.0,0.0,16.93,1610528863
61,-27.198298,147.966471,roma,au,84.20,31.0,0.0,9.31,1610528865
84,9.507956,-8.297668,odienne,ci,77.77,18.0,0.0,4.76,1610528871
85,15.436669,-139.951558,hilo,us,69.80,78.0,1.0,6.91,1610528660


## Hotel Map

In [150]:
# Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.
# Store into variable named hotel_df.

hotel_df = ideal_weather.copy()

# resetting index
hotel_df = hotel_df.reset_index(drop=True)

#Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""

hotel_df

,Latitude,Longitude,City,Country,Max Temperature (F),Humidity,Cloudiness,Wind Speed (mph),Date,Hotel Name
0,-14.936039,13.322562,lubango,ao,69.80,56.0,0.0,3.44,1610528669,
1,-35.580137,165.168906,ahipara,nz,65.88,84.0,10.0,6.15,1610528850,
2,-56.473162,104.629318,busselton,au,81.00,36.0,0.0,1.01,1610528771,
3,-53.198595,-17.768423,arraial do cabo,br,75.20,88.0,0.0,12.66,1610528852,
4,-33.583648,130.542010,flinders,au,73.99,79.0,10.0,8.99,1610528854,
5,34.992028,-153.096680,kapaa,us,73.40,83.0,1.0,6.91,1610528753,
6,-65.254553,-129.726240,rikitea,pf,77.76,79.0,0.0,16.93,1610528863,
7,-27.198298,147.966471,roma,au,84.20,31.0,0.0,9.31,1610528865,
8,9.507956,-8.297668,odienne,ci,77.77,18.0,0.0,4.76,1610528871,
9,15.436669,-139.951558,hilo,us,69.80,78.0,1.0,6.91,1610528660,


In [151]:
# Using the Google Places API for each city's coordinates.
# Storing the first Hotel result into the DataFrame.


# Setting empty hotel name list
hotels = []

# Base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


for index, row in hotel_df.iterrows():

    # setting parameters and geocoordinates to search for hotels with 5000 meters.
    city = row['City']
    lat = row['Latitude']
    long = row['Longitude']
    locations = f"{lat},{lng}"
    target_search = "hotel"
    target_radius = 5000
    target_type = "lodging"
                              
    # Creating a parameters dictionary
    params = {"location": locations, 
              "keyword": target_search, 
              "radius": target_radius, 
              "type": target_type, 
              "key": g_key}                            
      
    print("--------------------------------------")  
    print(f"Retrieving Results for Index: {city}.")
    
    # adding to data frame using try and except incase no data    
    try:    
    # running a request using our params dictionary
        response = requests.get(base_url, params=params)

    # converting response to json
        hotel_data = response.json()    
     
    # appending the hotel to the hotel list    
        hotels.append(hotel_data["results"][0]["name"])    
    
        print("Results Found")
    
    except IndexError:
        print("Unable to retrieve results... skipping.")
        hotels.append(np.nan)    
    
#adding list of hotels to data frame
hotel_df["Hotel Name"] = hotels


--------------------------------------
Retrieving Results for Index: lubango.
Unable to retrieve results... skipping.
--------------------------------------
Retrieving Results for Index: ahipara.
Unable to retrieve results... skipping.
--------------------------------------
Retrieving Results for Index: busselton.
Unable to retrieve results... skipping.
--------------------------------------
Retrieving Results for Index: arraial do cabo.
Unable to retrieve results... skipping.
--------------------------------------
Retrieving Results for Index: flinders.
Unable to retrieve results... skipping.
--------------------------------------
Retrieving Results for Index: kapaa.
Unable to retrieve results... skipping.
--------------------------------------
Retrieving Results for Index: rikitea.
Unable to retrieve results... skipping.
--------------------------------------
Retrieving Results for Index: roma.
Unable to retrieve results... skipping.
--------------------------------------
Retrieving 

In [153]:
# Cleaning the dataframe to remove rows with no hotel found
hotel_df = hotel_df.dropna()
hotel_df

,Latitude,Longitude,City,Country,Max Temperature (F),Humidity,Cloudiness,Wind Speed (mph),Date,Hotel Name
14,23.349151,-76.664179,rock sound,bs,75.60,76.0,13.0,7.94,1610528879,Thank Inn Plus Hotel Yunnan Honghe Gejiu City ...
25,19.276949,111.137525,wanning,cn,65.39,54.0,0.0,9.37,1610528916,Ban Napia - Plain of Jars homestay
28,17.209428,80.409976,khammam,in,89.58,34.0,12.0,3.38,1610528919,James & oi homestay
33,11.609698,-96.586869,pochutla,mx,72.00,75.0,0.0,1.99,1610528939,Rainbow Lodge
38,26.115415,84.579127,maharajganj,in,69.80,39.0,0.0,4.14,1610528947,Xintairan Hostel
47,17.212518,103.140275,nong han,th,78.01,31.0,0.0,3.44,1610528964,James & oi homestay


In [156]:
# Using the template to add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]




In [157]:
#Plotting markers on top of the heatmap.

# Assigning the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)


# Adding the layer to the map
fig.add_layer(markers)

# Displaying the map
fig

Figure(layout=FigureLayout(height='420px'))